In [1]:
import numpy as np
import pylab as pl
import torch
import torch.optim as optim
import torch.nn as nn
import random
from collections import OrderedDict

torch.set_printoptions(edgeitems=2)

In [2]:
# 定义拟合网络：将输入的变量映射到最后的值：
class FitModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
                nn.Linear(100, 300),
                nn.ReLU(),
                nn.Linear(300, 500),
                nn.ReLU(),
                nn.Linear(500, 300),
                nn.ReLU(),
                nn.Linear(300, 1)
            )
        
        
    def forward(self, x):
        x = self.net(x)
        return x

# 将定义的模型实例化：
fitNet = FitModel()
# 为实例化后的模型定义优化器
fitOpt = optim.SGD(fitNet.parameters(), lr=0.01)

In [3]:
# 定义生成网络：生成100个输入值：

# 由于输入值只能为1或-1，所以选取tanh函数，通过对x放大，可以将实数域的值
# 映射到近似1或-1两个值（可以绘制tanh(100x)的图形帮助理解）
# 于是定义如下激活函数：
class MyTanh(nn.Module):
    def __init__(self):
        super(MyTanh, self).__init__()
        
    def forward(self, x):
        x = torch.tanh(100*x)
        return x 

# 接下来定义生成网络：
class GenModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 网络1：得出输入        
        self.net = nn.Sequential(
                nn.Linear(100, 300),
                nn.Tanh(),
                nn.Linear(300, 500),
                nn.Tanh(),
                nn.Linear(500, 600),
                nn.LeakyReLU(),
                nn.Linear(600, 500),
                nn.LeakyReLU(),
                nn.Linear(500, 300),
                nn.LeakyReLU(),
                nn.Linear(300, 100),
                MyTanh()
            )
        
    def forward(self, x):
        x = self.net(x)
        return x

genNet = GenModel()
genOpt = optim.SGD(genNet.parameters(), lr=0.05)

In [4]:
# 输出获取函数
def getOutput(inList):
    return random.randint(0, 10)

# 输入输出获取函数
def generateInput(size):
    inputList = []
    outputList=[]
    i=0
    while i<size: 
        inList = []
        for j in range(100):
            item = random.randint(0, 1)
            if item == 0:
                item = -1
            inList.append(item)
        outList=getOutput(inList)
        i=i+1
        inputList.append(inList)
        outputList.append(outList)
        
    return inputList,outputList

In [5]:
# 获取输入数据与输出数据：
inputList,outputList = generateInput(500)

In [6]:
# 对输入数据与输出数据tensor包装，以便输入神经网络进行训练：
inputList = torch.tensor(inputList)
outputList = torch.tensor(outputList).unsqueeze(1)

# 进行训练集与测试集划分：
n_samples = inputList.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

input_train = inputList[train_indices]     # 训练集：输入数据
output_train = outputList[train_indices]   # 训练集：输出数据

input_val = inputList[val_indices]         # 测试集：输入数据
output_val = outputList[val_indices]       # 测试集：输出数据

In [7]:
# 定义拟合网络的训练函数：
def fitTrain(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 50 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(loss_train), float(loss_val)))

In [8]:
# 进行拟合函数的训练：
fitTrain(
    n_epochs = 200, 
    optimizer = fitOpt,
    model = fitNet,
    loss_fn = nn.MSELoss(),
    t_u_train = input_train.float(),
    t_u_val = input_val.float(), 
    t_c_train = output_train.float(),
    t_c_val = output_val.float())

Epoch 1, Training loss 36.51694869995117, Validation loss 39.02036666870117
Epoch 50, Training loss 5.6283488273620605, Validation loss 10.69279670715332
Epoch 100, Training loss 4.953967094421387, Validation loss 18.365276336669922
Epoch 150, Training loss 1.0587949752807617, Validation loss 15.169242858886719
Epoch 200, Training loss 0.42391857504844666, Validation loss 14.633149147033691


In [9]:
# 拟合网络训练结束后，冻结其全部参数，以免在生成网络的训练过程中被修改：
fitNet.require_grad = False

In [10]:
# 定义生成网络的训练：
def genTrain(n_epochs, optimizer, model):
    val = torch.ones(100)
    out = 0
    for epoch in range(1, n_epochs+1):
        out = model(val)
        
        # 我们希望fitNet(out)的值越大越好
#         loss = torch.exp(1/fitNet(out))
        loss = -fitNet(out)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 500 == 0:
            print('Epoch {}, loss {}'.format(
                epoch, float(loss)))
    return out

In [11]:
# 进行生成网络的训练：
out = genTrain(2500, genOpt, genNet)

Epoch 1, loss -5.791012287139893
Epoch 500, loss -7.239447116851807
Epoch 1000, loss -7.4095683097839355
Epoch 1500, loss -7.4095683097839355
Epoch 2000, loss -7.4095683097839355
Epoch 2500, loss -7.4095683097839355


In [12]:
out

tensor([-1., -1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
         1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
         1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
        -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
         1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
         1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1.,
        -1., -1.], grad_fn=<TanhBackward>)